<a href="https://colab.research.google.com/github/devjmc/music_investigation/blob/master/DeepbachTry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
# This is formatted as code
```
# Using DeepBach in colab.
The intention is to use a GAN for music composition.

1)git clone repository and run dl_dataset_and_models.sh. See readme in github repo.

In [2]:
!git clone -l -s git://github.com/Ghadjeres/DeepBach.git

Cloning into 'DeepBach'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 566 (delta 2), reused 3 (delta 1), pack-reused 555
Receiving objects: 100% (566/566), 222.39 MiB | 16.13 MiB/s, done.
Resolving deltas: 100% (320/320), done.


In [3]:
%cd DeepBach

/content/DeepBach


In [4]:
#@title
!ls

DatasetManager	       dl_dataset_and_models.sh  flask_server.py
DeepBach	       Dockerfile		 LICENSE
deepBachMuseScore.qml  entrypoint.sh		 musescore_flask_server.py
deepBach.py	       environment.yml		 README.md


In [5]:
!bash dl_dataset_and_models.sh

--2020-05-28 12:20:15--  https://www.dropbox.com/s/iuz4ml857ycyyat/deepbach_pytorch_resources.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:6021:1::a27d:4101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/iuz4ml857ycyyat/deepbach_pytorch_resources.tar.gz [following]
--2020-05-28 12:20:15--  https://www.dropbox.com/s/raw/iuz4ml857ycyyat/deepbach_pytorch_resources.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc24a54bfe43ba17272b47932ee5.dl.dropboxusercontent.com/cd/0/inline/A4klWsJEocjIX-hKsQHjyps2rVhJwEXX0_1-uj2Pdz9gvqGHdqVHT419HLGLp_t6k3qL23wHVJQPnIW0bZdkpiHyeHLYBH0zbVNTd0HRnLJx_g/file# [following]
--2020-05-28 12:20:16--  https://uc24a54bfe43ba17272b47932ee5.dl.dropboxusercontent.com/cd/0/inline/A4klWsJEocjIX-hKsQHjyps2rVhJwEXX0_1-uj2Pdz9gvqGHdqVHT419HLGLp_t6k3qL2

In [6]:
!pwd

/content/DeepBach


In [0]:
import sys
sys.path.append('/content/DeepBach')

In [8]:
!ls

DatasetManager			   entrypoint.sh
DeepBach			   environment.yml
deepBachMuseScore.qml		   flask_server.py
deepBach.py			   LICENSE
deepbach_pytorch_resources.tar.gz  models
dl_dataset_and_models.sh	   musescore_flask_server.py
Dockerfile			   README.md


In [0]:
import click

from DatasetManager.chorale_dataset import ChoraleDataset
from DatasetManager.dataset_manager import DatasetManager
from DatasetManager.metadata import FermataMetadata, TickMetadata, KeyMetadata

from DeepBach.model_manager import DeepBach

def mymain(note_embedding_dim=20, meta_embedding_dim=20, num_layers=2, lstm_hidden_size=256, dropout_lstm=0.5, linear_hidden_size=256, batch_size=256, num_epochs=5, train=False, num_iterations=500, sequence_length_ticks=64  
         ):
    dataset_manager = DatasetManager()

    metadatas = [
       FermataMetadata(),
       TickMetadata(subdivision=4),
       KeyMetadata()
    ]
    chorale_dataset_kwargs = {
        'voice_ids':      [0, 1, 2, 3],
        'metadatas':      metadatas,
        'sequences_size': 8,
        'subdivision':    4
    }
    bach_chorales_dataset: ChoraleDataset = dataset_manager.get_dataset(
        name='bach_chorales',
        **chorale_dataset_kwargs
        )
    dataset = bach_chorales_dataset

    deepbach = DeepBach(
        dataset=dataset,
        note_embedding_dim=note_embedding_dim,
        meta_embedding_dim=meta_embedding_dim,
        num_layers=num_layers,
        lstm_hidden_size=lstm_hidden_size,
        dropout_lstm=dropout_lstm,
        linear_hidden_size=linear_hidden_size
    )

    if train:
        deepbach.train(batch_size=batch_size,
                       num_epochs=num_epochs)
    else:
        deepbach.load()
        deepbach.cuda()

    print('Generation')
    score, tensor_chorale, tensor_metadata = deepbach.generation(
        num_iterations=num_iterations,
        sequence_length_ticks=sequence_length_ticks,
    )
    score.show('txt')
    #score.show()

In [10]:
# train=True is failing with "KeyError with chorale 306"
mymain(train=True)

Loading ChoraleDataset([0, 1, 2, 3],bach_chorales,['fermata', 'tick', 'key'],8,4) from /content/DeepBach/DatasetManager/dataset_cache/datasets/ChoraleDataset([0, 1, 2, 3],bach_chorales,['fermata', 'tick', 'key'],8,4)
(the corresponding TensorDataset is not loaded)


0it [00:00, ?it/s]

===Epoch 0===
Creating ChoraleDataset([0, 1, 2, 3],bach_chorales,['fermata', 'tick', 'key'],8,4) TensorDataset since it is not cached
Making tensor dataset
Computing index dicts


351it [00:57,  6.08it/s]
4it [00:00, 868.52it/s]
0it [00:00, ?it/s]

Computing voice ranges


...

307it [2:32:51, 1048.96s/it]

KeyError with chorale 306


351it [2:45:51, 28.35s/it]


Sizes: torch.Size([1065661, 4, 32]), torch.Size([1065661, 4, 32, 4])
TensorDataset for ChoraleDataset([0, 1, 2, 3],bach_chorales,['fermata', 'tick', 'key'],8,4) saved in /content/DeepBach/DatasetManager/dataset_cache/tensor_datasets/ChoraleDataset([0, 1, 2, 3],bach_chorales,['fermata', 'tick', 'key'],8,4)


RuntimeError: ignored

Error message during training. 
It appears it is due during the saving of the results.

307it [2:32:51, 1048.96s/it]KeyError with chorale 306
351it [2:45:51, 28.35s/it]
Sizes: torch.Size([1065661, 4, 32]), torch.Size([1065661, 4, 32, 4])
TensorDataset for ChoraleDataset([0, 1, 2, 3],bach_chorales,['fermata', 'tick', 'key'],8,4) saved in /content/DeepBach/DatasetManager/dataset_cache/tensor_datasets/ChoraleDataset([0, 1, 2, 3],bach_chorales,['fermata', 'tick', 'key'],8,4)
---------------------------------------------------------------------------
RuntimeError                              Traceback (most recent call last)
<ipython-input-10-81543a695b96> in <module>()
----> 1 mymain(train=True)

4 frames
<ipython-input-9-0979c21202a6> in mymain(note_embedding_dim, meta_embedding_dim, num_layers, lstm_hidden_size, dropout_lstm, linear_hidden_size, batch_size, num_epochs, train, num_iterations, sequence_length_ticks)
     40     if train:
     41         deepbach.train(batch_size=batch_size,
---> 42                        num_epochs=num_epochs)
     43     else:
     44         deepbach.load()

/content/DeepBach/DeepBach/model_manager.py in train(self, main_voice_index, **kwargs)
     68         if main_voice_index is None:
     69             for voice_index in range(self.num_voices):
---> 70                 self.train(main_voice_index=voice_index, **kwargs)
     71         else:
     72             voice_model = self.voice_models[main_voice_index]

/content/DeepBach/DeepBach/model_manager.py in train(self, main_voice_index, **kwargs)
     74                 voice_model.cuda()
     75             optimizer = optim.Adam(voice_model.parameters())
---> 76             voice_model.train_model(optimizer=optimizer, **kwargs)
     77 
     78     def eval_phase(self):

/content/DeepBach/DeepBach/voice_model.py in train_model(self, batch_size, num_epochs, optimizer)
    222             loss, acc = self.loss_and_acc(dataloader_train,
    223                                           optimizer=optimizer,
--> 224                                           phase='train')
    225             print(f'Training loss: {loss}')
    226             print(f'Training accuracy: {acc}')

/content/DeepBach/DeepBach/voice_model.py in loss_and_acc(self, dataloader, optimizer, phase)
    273                                 target=label)
    274 
--> 275             average_loss += loss.item()
    276             average_acc += acc.item()
    277 

RuntimeError: CUDA error: an illegal memory access was encountered